# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews


In [4]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

,reviews
0,Not Verified | I travelled with British Airway...
1,Not Verified | Food was lousy. Who ever is pl...
2,✅ Trip Verified | Had the worst experience. Th...
3,✅ Trip Verified | The ground staff were not h...
4,✅ Trip Verified | Second time BA Premium Econ...


In [5]:
df.to_csv("data/BA_reviews.csv")


Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [3]:
df = pd.read_csv("data/BA_reviews.csv")

In [4]:
df = pd.DataFrame(df["reviews"].str.split('|').tolist(), columns=["trip_verification","reviews"])

In [5]:
df.replace({'❎ Not Verified ': False, '✅ Trip Verified ': True, 'Not Verified ':False}, inplace=True)

In [6]:
df

,trip_verification,reviews
0,False,I travelled with British Airways from Sweden ...
1,False,Food was lousy. Who ever is planning the Asi...
2,True,Had the worst experience. The flight from Lon...
3,True,The ground staff were not helpful. Felt like...
4,True,Second time BA Premium Economy in a newer ai...
...,...,...
995,True,Mumbai to London Heathrow. Disappointing exp...
996,True,Delhi to London. Having read many negative r...
997,True,When you travel British Airways its like you ...
998,True,British Airways gets plenty of well deserved...


In [7]:
df.trip_verification.value_counts()

True     814
False    186
Name: trip_verification, dtype: int64

In [8]:
#Removal of stop words from the reviews
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words


In [9]:
#adding common words which do not contribute to the sentiment of the review to the stope words
set(list(sw_spacy)+['flight','BA','British Airway','London'])



{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'BA',
 'British Airway',
 'London',
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'flight',
 'for',
 'former',

In [10]:
df['reviews'] = df['reviews'].str.split().tolist()

In [11]:
df['reviews'] = df['reviews'].apply(lambda x: [item for item in x if item not in sw_spacy])

In [12]:
df
#The stop words are now removed from the reviews

,trip_verification,reviews
0,False,"[I, travelled, British, Airways, Sweden, Los, ..."
1,False,"[Food, lousy., Who, planning, Asian, Hindu, Ve..."
2,True,"[Had, worst, experience., The, flight, London,..."
3,True,"[The, ground, staff, helpful., Felt, like, wan..."
4,True,"[Second, time, BA, Premium, Economy, newer, ai..."
...,...,...
995,True,"[Mumbai, London, Heathrow., Disappointing, exp..."
996,True,"[Delhi, London., Having, read, negative, revie..."
997,True,"[When, travel, British, Airways, like, lucky, ..."
998,True,"[British, Airways, gets, plenty, deserved, kic..."


In [13]:
#Lemmatizing the reviews
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()
df['reviews'] = df['reviews'].apply(lambda x: [wnl.lemmatize(item, pos='v') for item in x])
df['reviews'] = df['reviews'].apply(lambda x: [wnl.lemmatize(item, pos='r') for item in x])
df['reviews'] = df['reviews'].apply(lambda x: [wnl.lemmatize(item, pos='n') for item in x])


[nltk_data] Downloading package wordnet to /home/shashank/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
before_lem_len = df['reviews'].apply(lambda x: len(x))

In [15]:
df['reviews'].apply(lambda x: list(set(x)))
after_lem_len = df['reviews'].apply(lambda x: len(x))

In [16]:
diff = np.array(before_lem_len) - np.array(after_lem_len)
diff.sum()


0

In [17]:
df['num_words'] = df['reviews'].apply(lambda x: len(x))


In [18]:
df['review_sentence'] = df['reviews'].apply(lambda x: ' '.join(x))

In [19]:
df.head()

,trip_verification,reviews,num_words,review_sentence
0,False,"[I, travel, British, Airways, Sweden, Los, Ang...",196,I travel British Airways Sweden Los Angeles Lo...
1,False,"[Food, lousy., Who, plan, Asian, Hindu, Vegeta...",54,Food lousy. Who plan Asian Hindu Vegetarian me...
2,True,"[Had, worst, experience., The, flight, London,...",59,Had worst experience. The flight London Toront...
3,True,"[The, grind, staff, helpful., Felt, like, want...",32,The grind staff helpful. Felt like want rush c...
4,True,"[Second, time, BA, Premium, Economy, newer, ai...",50,Second time BA Premium Economy newer aircraft ...


In [20]:
df.to_csv("data/BA_reviews_lemmatized.csv", index=False)

In [24]:
df['reviews'][5]

['They',
 'change',
 'Flights',
 'Brussels',
 'London',
 'Heathrow',
 'LAX',
 '4/16/2023.',
 'We',
 'pay',
 'extra',
 'choose',
 'seats.',
 'Since',
 'cancel',
 'honor',
 'seat',
 'bought,',
 'seat',
 'totally',
 'different',
 'seats.',
 'I',
 'ask',
 'check',
 'employee,',
 'rude',
 'tell',
 'understand',
 'different',
 'flight.',
 'From',
 'London',
 'LAX',
 'worse,',
 'airport',
 'help',
 'us.',
 'Employees',
 'BA',
 'tell',
 'return',
 'day',
 'flight',
 'rent',
 'hotel',
 'terminal',
 '3',
 'sleep',
 'there.',
 'Finally',
 'employee',
 'help',
 'give',
 'voucher',
 'hotel.',
 'It',
 'nightmare',
 'airline.',
 'We',
 'miss',
 'day',
 'work',
 'BA',
 'didn’t',
 'return',
 'money',
 'pay',
 'previous',
 'choose',
 'seats.']

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shashank/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Topic: 0 
Words: 0.018*"BA" + 0.016*"I" + 0.014*"flight" + 0.011*"The" + 0.011*"seat" + 0.009*"business" + 0.008*"class" + 0.007*"fly" + 0.006*"London" + 0.006*"service"
Topic: 1 
Words: 0.038*"I" + 0.015*"flight" + 0.012*"BA" + 0.010*"book" + 0.008*"cancel" + 0.006*"We" + 0.006*"British" + 0.006*"The" + 0.005*"airline" + 0.004*"Airways"
Topic: 2 
Words: 0.014*"The" + 0.013*"I" + 0.010*"seat" + 0.008*"BA" + 0.008*"food" + 0.006*"flight" + 0.005*"good" + 0.005*"service" + 0.005*"crew" + 0.004*"time"
Topic: 3 
Words: 0.030*"I" + 0.012*"BA" + 0.011*"seat" + 0.008*"flight" + 0.008*"pay" + 0.007*"The" + 0.006*"time" + 0.006*"London" + 0.005*"try" + 0.005*"hour"
Topic: 4 
Words: 0.060*"I" + 0.024*"flight" + 0.012*"The" + 0.010*"BA" + 0.008*"hour" + 0.007*"tell" + 0.007*"service" + 0.006*"London" + 0.006*"seat" + 0.006*"fly"
Topic: 5 
Words: 0.019*"flight" + 0.013*"I" + 0.012*"The" + 0.010*"British" + 0.007*"service" + 0.007*"BA" + 0.007*"seat" + 0.007*"Airways" + 0.005*"check" + 0.005*"food"